In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import drive
import os

# Mount Google Drive
#drive.mount('/content/drive')

# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder, '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Data')  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['ES 09-24_Minute_5.csv',
 'NQ 09-24_Minute_5.csv',
 'ZB 09-24_Minute_5.csv',
 '.ipynb_checkpoints']

In [19]:
file_path_5min = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Data/ES 09-24_Minute_5.csv'


In [20]:
ES_5_Min = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Data/ES 09-24_Minute_5.csv")
pd.set_option('display.max_columns',40)
ES_5_Min.head()

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target
0,06/23/2019 18:20,3235.50,3235.50,3234.75,3235.50,0.0,0.0,0.0,0.0,0.00,3230.0,0.00,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,06/23/2019 18:25,3235.50,3235.75,3234.50,3234.50,0.0,0.0,0.0,0.0,3235.75,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,06/23/2019 18:30,3234.75,3235.75,3234.50,3235.75,0.0,0.0,0.0,0.0,3235.75,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,06/23/2019 18:35,3235.75,3236.50,3235.25,3236.50,0.0,0.0,0.0,0.0,0.00,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,06/23/2019 18:40,3236.50,3236.50,3235.75,3236.00,0.0,0.0,0.0,0.0,0.00,3230.0,3234.25,3235.75,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [21]:
print(ES_5_Min.shape)
print(ES_5_Min.dtypes)
print(ES_5_Min.isnull().sum())

(505221, 23)
Time Stamp                object
Open                     float64
High                     float64
Low                      float64
Close                    float64
SellersLevel4            float64
SellersLevel3            float64
SellersLevel2            float64
SellersLevel1            float64
SellersLevel0            float64
BuyersLevel0             float64
BuyersLevel1             float64
BuyersLevel2             float64
BuyersLevel3             float64
BuyersLevel4             float64
BuyersInControlTime       object
BuyersInControlPrice     float64
SellersInControlTime      object
SellersInControlPrice    float64
Entry                    float64
Defence                  float64
Stop                     float64
Target                   float64
dtype: object
Time Stamp               0
Open                     0
High                     0
Low                      0
Close                    0
SellersLevel4            0
SellersLevel3            0
SellersLevel2            

In [22]:
ES_5_Min.drop(columns=["Entry","Defence",	"Stop","Target"],axis=1,inplace=True)

In [23]:
ES_5_Min.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice'],
      dtype='object')

In [24]:
def generate_trading_features(df):
    # Initialize the new columns with zeros
    df['Entry'] = 0
    df['Target_b'] = 0
    df['Target_s'] = 0
    df['Defence_buyer'] = 0
    df['Defence_sale'] = 0
    df['Stop_buyer'] = 0
    df['Stop_sale'] = 0

    # Iterate over each row to calculate the features
    for index, row in df.iterrows():
        buyers_entry = 0
        sellers_entry = 0
        target_b = 0
        target_s = 0
        defence_buyer = 0
        defence_sale = 0
        stop_buyer = 0
        stop_sale = 0

        if pd.notna(row['BuyersInControlPrice']) and row['BuyersInControlPrice'] != 0:
            buyers_entry = row['BuyersInControlPrice']
            target_b = buyers_entry + 2
            defence_buyer = buyers_entry - 1  # Default defense for buyer
            stop_buyer = buyers_entry - 2     # Default stop for buyer

        if pd.notna(row['SellersInControlPrice']) and row['SellersInControlPrice'] != 0:
            sellers_entry = row['SellersInControlPrice']
            target_s = sellers_entry - 2
            defence_sale = sellers_entry + 1  # Modified defense for seller
            stop_sale = sellers_entry + 2     # Modified stop for seller

        # Use the non-zero value as the entry
        entry = buyers_entry if buyers_entry != 0 else sellers_entry

        df.at[index, 'Entry'] = entry

        # Determine the defence and stop values
        df.at[index, 'Defence'] = defence_buyer if buyers_entry != 0 else defence_sale
        df.at[index, 'Stop'] = stop_buyer if buyers_entry != 0 else stop_sale

        # Determine the target values
        df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0
        df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0

    # Create a new column for combined target
    df["Target"] = df["Target_b"] + df["Target_s"]

    # Drop the individual target columns
    df.drop(columns=["Defence_buyer", "Defence_sale", "Stop_buyer", "Stop_sale", "Target_b", "Target_s"], axis=1, inplace=True)

    return df



In [25]:
Es_5_Min_target = generate_trading_features(ES_5_Min)

<ipython-input-24-66f2a499ea5b>:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Entry'] = entry
<ipython-input-24-66f2a499ea5b>:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3237.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0
<ipython-input-24-66f2a499ea5b>:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0


In [ ]:
Es_5_Min_target[["Time Stamp","BuyersInControlTime"	,"BuyersInControlPrice"	,"SellersInControlTime"	,"SellersInControlPrice"	,"Entry"	,"Defence"	,"Stop","Target"]].sample(10)

,Time Stamp,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target
231593,10/18/2021 19:45,0,0.0,10/18/2021 19:15,4821.00,4821.00,4822.00,4823.00,4819.00
365718,02/03/2023 15:30,0,0.0,0,0.00,0.00,0.00,0.00,0.00
264656,02/14/2022 03:00,0,0.0,02/14/2022 02:15,4765.00,4765.00,4766.00,4767.00,4763.00
404855,06/23/2023 16:15,0,0.0,0,0.00,0.00,0.00,0.00,0.00
152847,01/06/2021 01:05,01/06/2021 00:15,4018.5,0,0.00,4018.50,4017.50,4016.50,4020.50
472211,02/22/2024 08:40,0,0.0,0,0.00,0.00,0.00,0.00,0.00
41429,11/22/2019 11:30,0,0.0,11/22/2019 10:45,3385.75,3385.75,3386.75,3387.75,3383.75
339269,10/31/2022 22:40,0,0.0,0,0.00,0.00,0.00,0.00,0.00
8066,07/24/2019 06:35,0,0.0,07/24/2019 03:45,3283.50,3283.50,3284.50,3285.50,3281.50
40244,11/19/2019 11:15,0,0.0,11/19/2019 10:45,3403.00,3403.00,3404.00,3405.00,3401.00
